# Udtræk tekst fra Lokalplaner
Henter tekst ud af PDF'er fra plansystem og lægger det i en PostgreSQL tabel.

## 1. Klargør tabeller i databasen

Hent alle lokalplaner fra plansystems-WFS til PostgreSQL database:

```bash
ogr2ogr -f PostgreSQL PG:"dbname=xx host=xx port=xx user=xx password=xx" WFS:"http://geoservice.plansystem.dk/wfs?version=1.0.0" pdk:lokalplan -lco SCHEMA=proj_lokalplan_dokument -lco GEOMETRY_NAME=the_geom -nln "lokalplan" -a_srs "EPSG:25832"
```

Lav tabel i databasen til at indsætte planid, status og tilhørende dokumenttekst
```sql
# Lav skema og tabel
create schema elasticsearch;

create table elasticsearch.lokalplan_dokument (
	gid serial primary key not null,
	planid int4,
	plannavn varchar,
	status varchar,
	document text
)

## indsæt plandata ind i lokalplan_dokument
insert into elasticsearch.lokalplan_dokument(planid, plannavn, status)
SELECT planid, plannavn, status
FROM job_plandatadk.lokalplan
where STATUS in ('V', 'F') and AKTUEL = true and komnr = 147 and doklink is not null;
```

## 2. Opdateringer
Lokalplaner kan have følgende statuser - kladde, forslag, vedtage og aflyst. Vi er kun interesseret at oprette, opdatere eller fjerne lokalplandokumenter i ElasticSearch indexet når der sker ændring i planstatus. Ligeledes skal de data der trækkes fra WFS tilrettes i PostgreSQL. Opdateringsfrekvensen er selvfølgelig afgørende for om en plan kan springe en status over og som udgangspunkt kan opdateringen sættes til at ske en gang om måneden. Følgende scenarier gør sig gældende: 
#### Oprettes
* Planer der ændre status fra kladde til forslag
* Planer med planid som ikke allrede eksisterer (springer status over pga. opdateringfrekvensen)

#### Opdateres
* Planer der ændre status fra forslag til vedtaget

#### Slettes
* Planer der ændre status til aflyst

planid vil være det samme for en plan der ændre status, men i tabellen angives hvilken plan som er gældende i kolonnen aktuel.



In [1]:
import requests
import io
import PyPDF2
import logging
import psycopg2
import textract
import os
from sqlalchemy import create_engine, Table, MetaData, update, select

/Users/andersbarfod/anaconda/envs/plandata/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
/Users/andersbarfod/anaconda/envs/plandata/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import sys
sys.path.append('/Users/andersbarfod/Documents/python/')
import connections as con

In [3]:
logging.basicConfig(filename='logfile.log', filemode='w')

https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [4]:
def download_pdf(url, folder):
    """
    Download pdf from url to folder
    """
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)
        logging.warning(e)
    pdf_name = url.split('/')[-1]
    path = folder + pdf_name
    with open(path, 'wb') as file:
        file.write(r.content)
    return pdf_name

In [5]:
def get_document(url, temp_folder='./', language='dan'):
    """
    Get pdf documents from URL and extract the text. 
    If the pdf doesn't contain text textract is used to 
    OCR scan the document, which entails temporary downloads of pdf's.
    The function uses logging of exceptions, so make sure to setop logging basic config.
    
    logging.basicConfig(filename='logfile.log', filemode='w')
    """
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)
        logging.warning(e)
    # Convert to in-memory binary streams and read pdf
    pdf_file = io.BytesIO(r.content)
    pdfReader = PyPDF2.PdfFileReader(pdf_file, strict=False)

    # Discerning the number of pages will allow us to parse through all the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    # The while loop will read each page
    while count < num_pages:
        try:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        except Exception as e:
            logging.warning(e)
            print(e, url)
    #If the belov returns as False, we run the OCR library textract to 
    # convert scanned/image based PDF files into text         
    if text != "":
        text = text
    else:
        try:
            # Download pdf to disk in order to use textract (might be possible to do in-memory)
            filename = download_pdf(url, temp_folder)
            text = textract.process(temp_folder + filename, method='tesseract', language=language)
            text = text.decode('utf-8')
            if os.path.exists(filename):
                os.remove(filename)
        except Exception as e:
            logging.warning(e)
            print(e, url)

    return text

## Output til PostgreSQL
Ovenstående `get_document()` funktions bruges til at opdatere `plan_dokument` tabellen med teksten fra lokalplanerne

In [6]:
engine = con.engine('production')

In [7]:
connection = engine.connect()

In [8]:
metadata = MetaData()

In [9]:
plan = Table('lokalplan_dokument', metadata, autoload=True, autoload_with=engine, schema='elasticsearch')

In [10]:
def update_table(komkode):
    sql = '''
        select PLANID, STATUS, doklink
        FROM job_plandatadk.lokalplan
        where STATUS in ('V', 'F') and AKTUEL = true and komnr = {} and doklink is not null
        '''.format(komkode)
    result_set = connection.execute(sql)  
    for r in result_set:  
        plan_update = plan.update().values(document=get_document(r['doklink'])).where(plan.c.planid == r['planid'] and plan.c.status == r['status'])
        engine.execute(plan_update)
        logging.info('Indlæst: %s', plan.c.planid)

In [11]:
if __name__ == "__main__":
    update_table(147)

'/Contents' https://dokument.plandata.dk/20_3287138_1516291087510.pdf
'/Contents' https://dokument.plandata.dk/20_3287138_1516291087510.pdf
'/Contents' https://dokument.plandata.dk/20_3287138_1516291087510.pdf
name 'os' is not defined https://dokument.plandata.dk/20_1062041_APPROVED_1298984717792.pdf
name 'os' is not defined https://dokument.plandata.dk/20_1108674_APPROVED_1293720247017.pdf
